<a href="https://colab.research.google.com/github/vuvh87/CS2225.CH1402/blob/master/FaceInMp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/vuvh87/CS2225.CH1402.git vra
!mkdir Video
!mkdir Dataset

Cloning into 'vra'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 77 (delta 41), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [2]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=4d6c3cd7a049a3ce9c1cb0c4c78709a502eb9ee0a4ae2db72a9bac9204509201
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [3]:
import face_recognition
import cv2
from google.colab.patches import cv2_imshow

In [17]:
import io
import base64
from IPython.display import HTML

def playvideo(filename):
    video = io.open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" controls>
                    <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                 </video>'''.format(encoded.decode('ascii')))
#Now play...
playvideo('/content/vra/video/seth_godin_interview_small.mp4')

In [25]:
input_video = cv2.VideoCapture("/content/vra/video/seth_godin_interview_small.mp4")

In [26]:
# Metadata from the input video
frames_per_second = int(input_video.get(cv2.CAP_PROP_FPS))
frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

print('Metadata from input video:',
      '\nFrames per second:', frames_per_second,
      '\nFrame width:', frame_width, 
      '\nFrame height:', frame_height)

Metadata from input video: 
Frames per second: 29 
Frame width: 1280 
Frame height: 720


In [27]:
codec = cv2.VideoWriter.fourcc(*'XVID')
video_writer = cv2.VideoWriter('/content/Video/output_video.mp4', 
                               codec, 
                               frames_per_second, 
                               (frame_width, frame_height))

In [28]:
# OpenCV Default config file for face detection
haar_file = '/content/vra/haarcascades/haarcascade_frontalface_default.xml'
  
# Root Folder used to store the face data images
datasets = '/content/Dataset' 

# define the size of images  
(width, height) = (130, 100)     
  
# Analyse the video training file 
face_cascade = cv2.CascadeClassifier(haar_file) 

In [29]:
# An array to hold the locations of faces that are detected on individual frames
face_locations = []

# A counter to keep track of the number of frames processed
count = 1

# Loop through all the frames in the video
while (count != 1000):
  # Read the video to retrieve individual frames. 'frame' will reference the inidivdual frames read from the video.
  ret, frame = input_video.read()

  # Check the 'ret' (return value) to see if we have read all the frames in the video to exit the loop
  if not ret:
    print('Processed all frames')
    break

  # Convert the image (frame) to RGB format as by default Open CV uses BGR format. 
  # This conversion is done as face_recognition and other libraries usually use RGB format.
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  
  # Get the coordinates in the image where a face is detected. Use the model 'cnn' after greater accuracy.
  face_locations = face_recognition.face_locations(rgb_frame, model='cnn')

  # Loop through the face locations array and draw a rectangle around each face that is detected in the frame
  for top, right, bottom, left in face_locations:
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

  # Write the frame to the output vide0
  video_writer.write(frame)
  
  #cv2_imshow(frame)
  
  # Print for every 50 frames processed
  if(count % 50 == 0):
    print('Processed', count, 'frames')

  count += 1

  # Write the frame to image
  faces = face_cascade.detectMultiScale(rgb_frame, 1.3, 4)
  for (x, y, w, h) in faces: 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) 
        face = rgb_frame[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height)) 
        cv2.imwrite('% s/% s.png' % (datasets, count), face_resize) 

# Release to close all the resources that we have opened for reading and writing video
input_video.release()
video_writer.release()
cv2.destroyAllWindows()

Processed 50 frames
Processed 100 frames
Processed all frames
